### Imports and Setup

In [3]:
import torch
import classiq

import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

import sys
sys.path.append("..") # Add the parent directory to the sys.path list

from models.leqm3 import linear_entanglement_r3_quantum_model
from models.qnn import execute_fn, post_process_fn, QNN

from scripts.helper import create_writer, write_train_results
from scripts.data_setup import create_dataloaders_from_folders
from scripts.data_transforms import input_transform, target_transform
from scripts.train import train
from scripts.test import test

In [4]:
## Authenticate Classiq
classiq.authenticate()

The current version of 'classiq' has been deprecated, and will not be supported as of 2024-01-18. Please run "pip install -U classiq" to upgrade the classiq SDK to the latest version.
/home/devilkillerag/document/python_pro/QNN-MNIST-Classification/qnn/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [5]:
## For setting up device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
device

'cpu'

In [6]:
## Clear Output Files
post_process_output_file = open("post_process_output.txt", "w")
print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
print("--------------------------------------------POST PROCESS OUTPUT--------------------------------------------------", file=post_process_output_file)
print("-----------------------------------------------------------------------------------------------------------------", file=post_process_output_file)
post_process_output_file.close()

test_loop_output_file = open("test_loop_output.txt", "w")
print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
print("-----------------------------------------------TEST LOOP OUTPUT--------------------------------------------------", file=test_loop_output_file)
print("-----------------------------------------------------------------------------------------------------------------", file=test_loop_output_file)
test_loop_output_file.close()

In [7]:
# HYPER PARAMETERS
_LEARNING_RATE = 1.0
BATCH_SIZE = 64
EPOCHS = 2

### Quantum Model

In [8]:
#  Create a Linear Entanglement Quantum Model for MNIST Data Classification with three linear entanglement layers of RXX, RYY, and RZZ.
quantum_model = linear_entanglement_r3_quantum_model()

In [9]:
quantum_program = classiq.synthesize(quantum_model)

In [10]:
# View Quantum Program on Classiq Platform
classiq.show(quantum_program)

Opening: https://platform.classiq.io/circuit/ca968b8e-89ea-4964-99cc-d2bf74f6df9b?version=0.33.0


### Quantum Neural Network

In [11]:
qnn = QNN(
    quantum_program=quantum_program,
    execute=execute_fn,
    post_process=post_process_fn,
)

gio: https://platform.classiq.io/circuit/ca968b8e-89ea-4964-99cc-d2bf74f6df9b?version=0.33.0: Operation not supported


In [12]:
# summary(model=qnn, input_size=(32, 16), verbose=0, col_names=["input_size", "output_size", "num_params", "trainable"], col_width=20, row_settings=["var_names"])

In [13]:
# choosing our loss function
loss_fn = nn.L1Loss()
# choosing our optimizer
optimizer = optim.SGD(qnn.parameters(), lr=_LEARNING_RATE)

### Preparing Data

In [ ]:
train_dir = ''

In [14]:
train_dataloader, test_dataloader, class_names = create_dataloaders_from_folders(
    root="data",
    transform=input_transform,
    target_transform=target_transform,
    batch_size=BATCH_SIZE,
    create_subset=True,
    subset_size=64
)

NameError: name 'create_dataloaders_from_folders' is not defined

In [ ]:
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")
print(f"Our Dataset have following classes: {class_names}")

In [ ]:
data, label = next(iter(train_dataloader))

print(f"Image shape: {data.shape} -> [batch_size, pixel_angle]")
print(f"Label shape: {label.shape} -> [batch_size, label_value]")

#### Run Experiment

##### 01. Train

In [ ]:
# Create a writer for tracking our experiment
writer = create_writer(experiment_name="data_0.1_percent", model_name="linear_entanglement_r3", extra=f"{EPOCHS}_epochs")

In [ ]:
train_results = train(
    model = qnn, 
    data_loader = train_dataloader, 
    loss_fn = loss_fn, 
    optimizer = optimizer, 
    writer = writer, 
    epochs = EPOCHS,
    device = device
)

In [ ]:
# Check out the model results
print(train_results)

In [ ]:
write_train_results(experiment_name="data_0.1_percent", model_name="linear_entanglement_r3", epochs=EPOCHS, results=train_results)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir runs

##### 02. Test

In [ ]:
test_results = test(
    model = qnn, 
    data_loader = test_dataloader,
    device = device
)

In [ ]:
print(test_results)

#### Save the trained model

In [ ]:
save_model(
    model=qnn,
    target_dir='outputs/saved_models',
    model_name='leqmr3_subset64'
)